In [5]:
import numpy as np

In [283]:
class Layer():
    
    def __init__(self,no_of_neurons,previous_layer=None,activation=None):
        # number of neurons in current layer
        self.n = no_of_neurons
        # layer that sends input into this layer
        self.previous_layer = previous_layer
        # layer that recieves this layer's output
        self.next_layer = None
        if (self.previous_layer is not None):
            # sort the hierarchy
            previous_layer.next_layer = self
            # initialize the weights between the layers
            previous_layer.initialize_weights()
            # set the activation function
            if (activation is None):
                self.set_activation()
            else:
                self.set_activation(activation)
        else:
            # this is the input layer
            self.set_input_layer()
    
    def initialize_weights(self):
        if (self.next_layer is not None):
            # weights
            self.weights = np.random.normal(0,0.1,(self.n,self.next_layer.n))
            # bias separately
            self.bias = np.random.normal(0,0.1,self.next_layer.n)
    
    def set_input_layer(self):
        self.func = lambda x:x
    
    def set_activation(self,func='ReLU'):
        if (func=='ReLU'):
            # set ReLU and the derivative
            self.func = ReLU
            self.deriv = dReLU
        else:
            self.func = func
            # TODO
            self.deriv = None

    def feed_forward(self,*z):
        output = self.func(z)
        if (self.next_layer is not None):
            return self.next_layer.feed_forward(output @ self.weights + self.bias)
        else:
            return output
    
    def update_weights(self,learning_rate,error):
        self.weights += learning_rate*error
        
    def back_propagate(self,*z):
        pass
            
def ReLU(x):
    # ReLU function
    return np.maximum(x,0)

def dReLU(x):
    # derivative of ReLU
    return np.maximum()

In [284]:
a = Layer(2)

In [285]:
#b = Layer(6,a)

In [286]:
c = Layer(1,a,activation=lambda x: x)

In [375]:
x = np.array((0,3))

error = x.sum() - a.feed_forward(x)[0][0][0]
error

0.00010099923179218706

In [376]:
a.weights

array([[0.82550504],
       [0.93048363]])

In [377]:
a.update_weights(0.1,error)